In [1]:
#import the essential libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Data: `Blobs` dataset

In [2]:
from sklearn.datasets import make_blobs

Generate a dataset with:
- 2000 samples with 10 features each;
- 8 classes of blobs (centers argument);
- cluster_std = 7

In [3]:
X, y = make_blobs(n_samples=2000, n_features=10, centers=8, cluster_std=7, random_state=0)

In [6]:
print(f'X.shape={X.shape}')
print(f'y.shape={y.shape}')

X.shape=(2000, 10)
y.shape=(2000,)
